# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**: Andrés Malvestiti

In [1]:
import numpy as np

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [2]:
example_matrix = np.array([[2,0,4,0,65], [6,3,4,1,8]])
# Norma l0
# Usando linalg
print('Norma L0:', np.linalg.norm(example_matrix, ord=0, axis=1))
# Norma l1
# Usando linalg
print('Norma L1:', np.linalg.norm(example_matrix, ord=1, axis=1))
# Norma l2
# Usando linalg
print('Norma L2:', np.linalg.norm(example_matrix, ord=2, axis=1))
# Norma l-inf
# Usando linalg
print('Norma L-inf:', np.linalg.norm(example_matrix, ord=np.inf, axis=1))

# Si hacemos los calculos con una funcion custom para una matriz
def norm(X, ord):
    """"Consideramos a X como una matrriz de 2x2 donde cada fila es un vector matematico"""
    if (ord == 0):
        return np.count_nonzero(X, axis=1)
    if (ord == np.inf):
        return np.max(X, axis=1)
    return (np.sum(np.abs(X) ** ord, axis=1) ** (1/ord))

print('Norma L0:', norm(example_matrix, ord=0))
print('Norma L-inf:', norm(example_matrix, ord=np.inf))
print('Norma L1:', norm(example_matrix, ord=1))
print('Norma L2:', norm(example_matrix, ord=2))

Norma L0: [3. 5.]
Norma L1: [71. 22.]
Norma L2: [65.15366452 11.22497216]
Norma L-inf: [65.  8.]
Norma L0: [3 5]
Norma L-inf: [65  8]
Norma L1: [71. 22.]
Norma L2: [65.15366452 11.22497216]


## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [3]:
truth = np.array([1,1,0,1,1,1,0,0,0,1])
prediction = np.array([1,1,1,1,0,0,1,1,0,0])

#print('TP', truth & prediction)
#print(np.sum(truth & prediction))
#print((truth == 1) & (prediction == 1))
#print('TP alt', np.sum((truth == 1) & (prediction == 1)))
#print('TN', np.sum((truth == 0) & (prediction == 0)))

TP = np.sum((truth == 1) & (prediction == 1))
TN = np.sum((truth == 0) & (prediction == 0))
FP = np.sum((truth == 0) & (prediction == 1))
FN = np.sum((truth == 1) & (prediction == 0))

precision = TP / (TP + FP)
recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print('TP: ', TP)
print('TN: ', TN)
print('FP: ', FP)
print('FN: ', FN)

print('Presicion: ', precision)
print('Recall: ', recall)
print('Accuracy: ', accuracy)


TP:  3
TN:  1
FP:  3
FN:  3
Presicion:  0.5
Recall:  0.5
Accuracy:  0.4


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [7]:
def split(X_input,
          Y_input,
          train_size=0.7,
          val_size=0.15,
          test_size=0.15,
          random_state=42,
          shuffle=True):
    
      # First we check the train/val/test sizes sum 100%
      # and X & Y have the same size
      # otherwise we throw an error 
      assert (train_size + val_size + test_size == 1)
      assert X_input.shape[0] == Y_input.shape[0]

      n = X_input.shape[0]
      # example [0.7, 0.15, 0.15]
      dataset_proportions = np.array([train_size, val_size, test_size])
      # we take all but the last one to do the cumulative sum
      # so the last part of the split takes the remaining elements
      # finally, for example if n is 10, then the split indices will be: [7, 8]
      split_indices = np.cumsum(dataset_proportions[:-1] * n).astype(int)

      if (shuffle):
            indices = np.random.choice(n, n, replace=False)
            train_indices = indices[0:split_indices[0],]
            validation_indices = indices[split_indices[0]:split_indices[1],]
            test_indices = indices[split_indices[1]:,]

            X_train_split = np.array([X_input[idx] for idx in train_indices])
            X_validation_split = np.array([X_input[idx] for idx in validation_indices])
            X_test_split = np.array([X_input[idx] for idx in test_indices])

            Y_train_split = np.array([Y_input[idx] for idx in train_indices])
            Y_validation_split = np.array([Y_input[idx] for idx in validation_indices])
            Y_test_split = np.array([Y_input[idx] for idx in test_indices])

      else:
            X_train_split = X_input[0:split_indices[0],]
            X_validation_split = X_input[split_indices[0]:split_indices[1],]
            X_test_split = X_input[split_indices[1]:,]

            Y_train_split = Y_input[0:split_indices[0],]
            Y_validation_split = Y_input[split_indices[0]:split_indices[1],]
            Y_test_split = Y_input[split_indices[1]:,]

      return [X_train_split, X_validation_split, X_test_split, Y_train_split, Y_validation_split, Y_test_split]


